# CAICT 風機結冰預測 - DL_CNN_TF

2017/07/24  
http://www.industrial-bigdata.com/competition/competitionAction!showDetail.action?competition.competitionId=1

### Outline

### Summary

## 1. Load Data and Modules

**Load Python modules:**

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import shutil
import csv
import sklearn as sk
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pylab import *
from IPython.core.interactiveshell import InteractiveShell


pd.set_option('display.max_columns', None)
InteractiveShell.ast_node_interactivity = "all"

** utils **

In [2]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  
        
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()

** load data**

In [3]:
def variable_selection(data, isTest=False):
    if isTest:
        fixed_variable = ['time','group']
    else:
        fixed_variable = ['label','event','time','group','timestamp']
    #selected_variable = ['pitch1_angle','pitch2_angle','pitch3_angle','pitch1_speed','pitch3_speed','pitch2_speed']
    selected_variable = ['pitch1_angle','pitch2_angle','pitch3_angle','pitch1_speed','pitch3_speed','pitch2_speed','power','yaw_position','environment_tmp']
    variable = selected_variable + fixed_variable 
    data = data[variable]
    return data

def load_resample_data(id, remove_unknown=True, split_case=20, split_validation=True, policy=1, varsel=False, label=None):
    filename = 'data/resample_data_p{POLICY}_{ID}.csv'.format(ID=id,POLICY=policy)
    data_resample = pd.read_csv(filename)
    if varsel:
        data_resample=variable_selection(data_resample)
    
    if remove_unknown:
        data_resample = data_resample[data_resample['label']!=2]
    
    if label is not None:
        data_resample = data_resample[data_resample['label']==label]
    
    data_resample_train = data_resample[data_resample['event']<split_case]
    data_resample_valid = data_resample[data_resample['event']>=split_case]

    train_aX = data_resample.drop(['label','time','group','timestamp','event'], axis=1)
    train_ay = data_resample['label']
    train_aY = np.eye(2)[train_ay.values.astype(int)]
    
    train_X = data_resample_train.drop(['label','time','group','timestamp','event'], axis=1)
    train_y = data_resample_train['label']
    valid_X = data_resample_valid.drop(['label','time','group','timestamp','event'], axis=1)
    valid_y = data_resample_valid['label']
    train_Y = np.eye(2)[train_y.values.astype(int)]
    valid_Y = np.eye(2)[valid_y.values.astype(int)]

    train_abnormal = data_resample_train[data_resample_train['label']==1]
    train_normal = data_resample_train[data_resample_train['label']==0]
    train_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(train_normal), j=len(train_normal)/len(data_resample_train),k=len(train_abnormal),m=len(train_abnormal)/len(data_resample_train), n=len(data_resample_train))
    
    valid_abnormal = data_resample_valid[data_resample_valid['label']==1]
    valid_normal = data_resample_valid[data_resample_valid['label']==0]
    valid_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(valid_normal), j=len(valid_normal)/len(data_resample_valid),k=len(valid_abnormal),m=len(valid_abnormal)/len(data_resample_valid), n=len(data_resample_valid))
    if split_validation:
        return data_resample, train_X, train_Y, train_y, valid_X, valid_Y, valid_y, train_desc, valid_desc
    else:
        return data_resample, train_aX, train_aY, train_ay

def load_test1_data(id,remove_unknown=True, dropna=True, varsel=False):
    filename = 'data/goodformat_{ID}.csv'.format(ID=id)
    data_raw = pd.read_csv(filename)
    if varsel:
        data_raw=variable_selection(data_raw)
        
    if dropna:
        data_raw = data_raw.dropna()
    else: 
        data_raw = data_raw.fillna(0)
        
    data_X = data_raw.drop(['label','time','group','timestamp'], axis=1).values
    data_y = data_raw['label'].values.astype(int)
    data_Y = np.eye(2)[data_y]
    
    data_abnormal = data_raw[data_raw['label']==1]
    data_normal = data_raw[data_raw['label']==0]
    data_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(data_normal), j=len(data_normal)/len(data_raw),k=len(data_abnormal),m=len(data_abnormal)/len(data_raw), n=len(data_raw))
    
    return data_raw, data_X, data_Y, data_y, data_desc

def load_test2_data(varsel=False):
    test_data = pd.read_csv('data/ice1/test/08/08_data.csv')
    if varsel:
        test_data=variable_selection(test_data,isTest=True)
    test_timeidx = test_data['time']
    tmp = test_data.drop(['time','group'], axis=1)
    test_X = tmp
    return test_data, test_X, test_timeidx


**scoring**

In [4]:
def myscore(true_y, pred_y):
    n,p =sk.metrics.confusion_matrix(true_y, pred_y)
    tn = n[0]
    fp = n[1]
    fn = p[0]
    tp = p[1]
    score = 1- 0.5*(fp/(tn+fp))- 0.5*(fn/(fn+tp)) 
    prec= precision_score(true_y, pred_y)
    recall = recall_score(true_y, pred_y)
    accuracy = accuracy_score(true_y,pred_y)
    confusions = (
    ('tn', tn),
    ('fn', fn),
    ('tp', tp),
    ('fp', fp)
    )
    msg = 'Score:{sco} \n\taccuracy:{acc}, \n\trecall:{rcl}, \n\tprecision:{psc}\n\t{cfmt}'.format(sco=score,acc=accuracy,rcl=recall,psc=prec,cfmt=confusions)
    return score, accuracy, prec, recall, OrderedDict(confusions),msg


**Load input data.** 

In [5]:
# Note: data is in DataFrame format 
train_raw, train_X, train_Y, train_y, valid_X, valid_Y, valid_y, train_desc, valid_desc = load_resample_data(id=15,policy=1,varsel=False)
valid_raw, valid_21_X, valid_21_Y, valid_21_y= load_resample_data(id=21,policy=0,split_validation=False,split_case=8,varsel=False)
test_raw, test_X,test_timeidx = load_test2_data(varsel=False)

In [6]:
print('15-train:', train_desc)
print('15-valid:',valid_desc)

15-train: 正常:35339 (0.61 percent), 結冰:22179 (0.39 percent), Total:57518
15-valid: 正常:7680 (0.64 percent), 結冰:4315 (0.36 percent), Total:11995


** 為了CNN需求, 更改 input X format **

In [7]:
def cnn_dataset(data, n_steps):
    dataset_new = []
    for idx, _ in enumerate(data):
        start_idx = idx-n_steps
        end_idx = idx+1
        if start_idx<0:
            continue
        n=np.concatenate(data[start_idx:end_idx], axis=0)
        dataset_new.append(n)
    dataset_new = np.asarray(dataset_new)
    return dataset_new

In [10]:
n_steps = 20
n_steps_=n_steps-1
train_X_new = cnn_dataset(train_X.values,n_steps_)
valid_X_new = cnn_dataset(valid_X.values,n_steps_)
valid_21_X_new = cnn_dataset(valid_21_X.values,n_steps_)
test_X = cnn_dataset(test_X.values,n_steps_)

In [17]:
test_X.shape

(202309, 520)

In [14]:
train_Y_new=train_Y[n_steps_:]
train_y_new=train_y[n_steps_:]
valid_Y_new=valid_Y[n_steps_:]
valid_y_new=valid_y[n_steps_:]
valid_21_Y_new=valid_21_Y[n_steps_:]
valid_21_y_new=valid_21_y[n_steps_:]
test_timeidx_new=test_timeidx[n_steps_:]

In [15]:
test_timeidx_new.shape

(202309,)

### 2. Modeling

** Parameters **

In [ ]:
tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")
Y_ =tf.placeholder(tf.float32, [None, n_classes], name="Y_")

** CNN Network Structure **

In [ ]:
def build_cnn():
    
    
    W_e_1 = weight_variable([784, 500], "w_e_1")
    b_e_1 = bias_variable([500], "b_e_1")
    h_e_1 = tf.nn.sigmoid(tf.add(tf.matmul(X, W_e_1), b_e_1))

### 3.1 Tensorflow 參數定義

In [102]:
# hyperparameters
n_inputs = len(train_X[1]) # 每一行的维度
n_hidden_unins = 13 # 中間RNN的hidden units
n_classes = len(train_Y[1])  # RNN最后的输出類別個数

tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")
Y_ =tf.placeholder(tf.float32, [None, n_classes], name="Y_")

W = {
    'in': tf.Variable(tf.random_uniform([n_inputs, n_hidden_unins], -1.0, 1.0), name="In_W"), # 输入层到中间层权重矩阵
    'out': tf.Variable(tf.random_uniform([n_hidden_unins, n_classes], -1.0, 1.0), name="Out_W"), # 中间层到输出层的权重
}

variable_summaries(W['in'],'In_W')
variable_summaries(W['out'],'Out_W')
B = {
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_unins]), name="In_Bias"), # 偏置
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes]), name="Out_Bias"),
}

variable_summaries(B['in'],'In_Bias')
variable_summaries(B['out'],'OUT_Bias')


### 3.2 網路結構 Network Structure

In [103]:
# hidden_layer for input
# X : (128, 28, 28)
with tf.name_scope("inLayer"):
    X_ = tf.reshape(X, [-1, n_inputs])  # [batch_size*28, 28]
    X_in = tf.matmul(X_, W['in']) + B['in'] # (-1, 28) matmul (28, hidden_units) => (-1, hidden_units)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_unins]) # (batch_size, steps, input_size)


In [106]:
?tf.contrib.rnn.GRUCell

In [116]:
with tf.name_scope("RNN_CELL"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_unins) 
    #rnn_outputs, rnn_states = tf.nn.dynamic_rnn(lstm_cell, X_in, dtype=tf.float32) 
    rnn_outputs, rnn_states = tf.nn.dynamic_rnn(lstm_cell, X_in, dtype=tf.float32) 
    

In [117]:
with tf.name_scope('outlayer'):
    _pred = tf.matmul(rnn_states[0], W['out']) + B['out']
    pred = tf.nn.softmax(_pred, name="pred")
    #rnn_outputs = tf.unstack(tf.transpose(rnn_outputs, [1,0,2]))  
    #pred = tf.matmul(rnn_outputs[-1], weights['out']) + biases['out'] 
    

### 3.3 誤差函數 Loss Function

In [118]:
with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=Y_))
    loss_scar = tf.summary.scalar('loss', loss)
    acc_scar = tf.summary.scalar('accuracy', accuracy)

### 3.4 Optimizer (gradient descent)

In [119]:
lr = 0.001 # learning rate
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)


### 3.5 開始訓練  Start Training

In [120]:
#先創造一個session, 然後記得要init variable
init = tf.global_variables_initializer()
merged_summary_op = tf.summary.merge_all()
sess = tf.InteractiveSession()
sess.run(init)

#if not os.path.exists('tb-log_1'):
#    os.mkdir('tb-log_1')
#summary_writer_train = tf.summary.FileWriter('tb-log1/train',graph=tf.get_default_graph())
#summary_writer_validation = tf.summary.FileWriter('tb-log1/validation',graph=tf.get_default_graph())

In [132]:
epoch = 10
batch_size = 100

total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X_new.shape[0], batch_size, replace=False)
        batch_x = train_X_new[rnd_idx]
        batch_y = train_Y_new[rnd_idx]
        batch_x = batch_x.reshape([batch_size, n_steps, n_inputs])
        _, loss_v1, acc_v1= sess.run([optimizer, loss, accuracy], feed_dict={X: batch_x, Y_:batch_y})
        #summary_writer_train.add_summary(summary, ep * total_batch + i)

    batch_vx = valid_X_new.reshape([-1, n_steps, n_inputs])
    loss_v2, acc_v2= sess.run([loss, accuracy], feed_dict={X: batch_vx , Y_: valid_Y_new})
    #summary_writer_validation.add_summary(summary, ep * total_batch + i)
    updateProgress('epoch:{x0}, batch:{x4}, train loss:{x1:.3f} acc:{x5:.3f}, test loss:{x3:.3f} acc:{x2:.3f}'.format(x0=ep,x2=round(acc_v2,3),x3=round(loss_v2,3),x4=i,x1=round(loss_v1,3), x5=round(acc_v1,3)))
    print()


KeyboardInterrupt: 

## 4. 預測與準確率評估  Evaluation 

#### Predict Training Data

In [133]:
# predict all X and get the accuracy
train_X_rs = train_X_new.reshape([-1, n_steps, n_inputs])
acc_t = accuracy.eval({X: train_X_rs , Y_: train_Y_new})
print('Accuracy:',acc_t)

pred_t = pd.DataFrame(pred.eval({X: train_X_rs}))
pred_t = pred_t.apply(np.argmax,axis=1)
pd.Series(train_y).value_counts()
pred_t.value_counts()

print("Precision", precision_score(train_y_new, pred_t))
print("Recall", sk.metrics.recall_score(train_y_new, pred_t))

s,_=myscore(train_y_new, pred_t)
print('score',s,_)

Accuracy: 0.9903


0    35339
1    22179
dtype: int64

0    35226
1    22093
dtype: int64

Precision 0.989363146698
Recall 0.985526849723
score 0.989419657075 {'tn': 34905, 'fn': 321, 'tp': 21858, 'fp': 235}


#### Predict  Validataion Data

In [134]:
# predict all X and get the accuracy
valid_X_rs = valid_X_new.reshape([-1, n_steps, n_inputs])
acc_v = accuracy.eval({X: valid_X_rs , Y_: valid_Y_new})
print('Accuracy:',acc_v)

pred_v = pd.DataFrame(pred.eval({X: valid_X_rs}))
pred_v = pred_v.apply(np.argmax,axis=1)
pred_v.value_counts()

print("Precision", precision_score(valid_y_new, pred_v))
print("Recall", sk.metrics.recall_score(valid_y_new, pred_v))

s,_=myscore(valid_y_new, pred_v)
print('score',s,_)

Accuracy: 0.673279


0    7363
1    4433
dtype: int64

Precision 0.551996390706
Recall 0.567091541136
score 0.650809505363 {'tn': 5495, 'fn': 1868, 'tp': 2447, 'fp': 1986}


** Predict another machine **

In [135]:
# predict all X and get the accuracy
valid_21_X_rs = valid_21_X_new.reshape([-1, n_steps, n_inputs])

valid_21_X_rs__1=valid_21_X_rs[:100000] 
valid_21_Y_new__1=valid_21_Y_new[:100000] 
valid_21_y_new__1 = valid_21_y_new[:100000] 
acc_t1 = accuracy.eval({X: valid_21_X_rs__1 , Y_: valid_21_Y_new__1})
print('Accuracy:',acc_t1)
pred_t1 = pd.DataFrame(pred.eval({X: valid_21_X_rs__1}))
pred_t1 = pred_t1.apply(np.argmax,axis=1)
pred_t1.value_counts()


valid_21_X_rs__2=valid_21_X_rs[100000:] 
valid_21_Y_new__2=valid_21_Y_new[100000:] 
valid_21_y_new__2 = valid_21_y_new[100000:] 

acc_t2 = accuracy.eval({X: valid_21_X_rs__2 , Y_: valid_21_Y_new__2})
print('Accuracy:',acc_t2)
pred_t2 = pd.DataFrame(pred.eval({X: valid_21_X_rs__2}))
pred_t2 = pred_t2.apply(np.argmax,axis=1)
pred_t2.value_counts()

pred_t = pred_t1.append(pred_t2)
pred_t.shape
pred_t.value_counts()

print("Precision", precision_score(valid_21_y_new, pred_t))
print("Recall", sk.metrics.recall_score(valid_21_y_new, pred_t))

s,_=myscore(valid_21_y_new, pred_t)
print('score',s,_)

Accuracy: 0.83332


0    85271
1    14729
dtype: int64

Accuracy: 0.901398


0    71886
1     7483
dtype: int64

(179369,)

0    157157
1     22212
dtype: int64

Precision 0.188096524401
Recall 0.392742996804
score 0.642931407369 {'tn': 150697, 'fn': 6460, 'tp': 4178, 'fp': 18034}


#### Predict Test Data

In [136]:
test_X_rs = test_X.reshape([-1, n_steps, n_inputs])
test_X_rs__1 = test_X_rs[:100000]
pred_T1 = pd.DataFrame(pred.eval({X: test_X_rs__1}))
pred_T1 = pred_T1.apply(np.argmax,axis=1)
pred_T1.value_counts()

test_X_rs__2 = test_X_rs[100000:]
pred_T2 = pd.DataFrame(pred.eval({X: test_X_rs__2}))
pred_T2 = pred_T2.apply(np.argmax,axis=1)
pred_T2.value_counts()



0    52413
1    47587
dtype: int64

0    85767
1    16362
dtype: int64

In [137]:
pred_T = pred_T1.append(pred_T2)
pred_T.shape
pred_T.value_counts()

(202129,)

0    138180
1     63949
dtype: int64

## 5. 整理預測結果, 正確上傳格式

In [138]:
pred_T.shape

(202129,)

In [139]:
startTime=0
endTIme=0
search_start=True
search_end=False
abnormal_list=[]
shift = test_timeidx_new.index[0]
for i,v in enumerate(pred_T):
    i=i+shift
    if (v==1) & (search_start):
        startTime = test_timeidx_new[i]
        search_end=True
        search_start=False
    if (v==0) & (search_end):
        endTIme=test_timeidx_new[i]
        search_start=True
        search_end=False
        abnormal_list.append((startTime,test_timeidx_new[i-1]))
len(abnormal_list)

1580

In [140]:
output_ans=True
if(output_ans):
    with open('test1_08_results.csv','w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['startTime','endTime'])
        for row in abnormal_list:
            dummy = csv_out.writerow(row)

print('done')


19

done


In [141]:
abnormal_list[:4]

[(1122.0, 1123.0), (1125.0, 1193.0), (1302.0, 1304.0), (1332.0, 1332.0)]

## 6. Issue
1. Data Imbalance
 - training batch sample 一半normal一半abnormal的data
 - 用兩台風機訓練兩個model, 取故障預測的聯集 

2. 用Precision & Recall評估好壞, 算出Score


other:
- using weighted examples. Just amplify the per-instance loss by a larger weight when seeing positive examples. If you use online gradient descent, it can be as simple as using a larger learning rate when seeing positive examples.
- A similar and slightly better approach (only if you use stochastic gradient descent) is randomly picking an example in each iteration, where the positive examples have higher probability of being picked.

提交结果历史记录  
参赛队伍:　556 / 参赛人数:　858  
 
|竞赛阶段	 |上传者	 |分数	 |提交日期	 |排名	 |下载|
| ---------| -------- | ------ | ------ | ------ | ------ |
|初赛test1阶段|	孔祥千	|55.28684214|	2017/7/12|	50	|下载|

Note: 7/13
1. cross correlation
2. auto correlation
3. common filter
4. de-train 